# city SVR 最終データ出力

#### 設定

In [1]:
# BLEUのインストール
!git clone https://github.com/yahoojapan/geobleu.git
!pip install geobleu/

fatal: destination path 'geobleu' already exists and is not an empty directory.
Processing ./geobleu
  Preparing metadata (setup.py) ... done
  Created wheel for geobleu: filename=geobleu-0.3-py3-none-any.whl size=5037 sha256=eef463804b6229a22fb963d3915b2e231eb0de486d07cdf8fe38c71d6d34053a
  Stored in directory: /tmp/pip-ephem-wheel-cache-q2kcmvo1/wheels/90/15/a1/d49421f5100216d00a75ab920989107243909c099af9364987
Successfully built geobleu
  Attempting uninstall: geobleu
    Found existing installation: geobleu 0.3
    Uninstalling geobleu-0.3:
      Successfully uninstalled geobleu-0.3


In [2]:
import pandas as pd
import numpy as np
import geobleu
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
!ls

 df_city_b_train_ip_cityB.csv
 df_city_b_train_ip_cityC.csv
 df_city_b_train_ip_cityD.csv
 future_predictions_df_cityB.csv
 future_predictions_df_cityC.csv
 future_predictions_df_cityD.csv
 geobleu
'humob-SVR_ver40_cityB_'$'\346\234\254\347\225\252''.ipynb'
'humob-SVR_ver40_cityC_'$'\346\234\254\347\225\252''.ipynb'
'humob-SVR_ver40_cityD_'$'\346\234\254\347\225\252''.ipynb'
'humob-SVR_'$'\343\203\207\343\203\274\343\202\277\346\225\264\345\275\242''.ipynb'


In [140]:
city = "C" # B,C,D

In [141]:
INPUT_PATH = f"../../../s3storage/01_public/humob-challenge-2024/input/city{city}_challengedata.csv.gz"
INPUT_PATH_future = f"future_predictions_df_city{city}.csv"

#### データ読み込み

In [150]:
df_city = pd.read_csv(INPUT_PATH)
df_city.head()

,uid,d,t,x,y
0,0,5,15,28,157
1,0,5,16,28,158
2,0,5,17,28,157
3,0,5,18,29,158
4,0,5,19,27,157


In [143]:
# あとで使うので解凍したcsvを出力
df_city.to_csv(f"df_city{city}.csv",index=False)

In [151]:
df_city_future = (
    pd.read_csv(INPUT_PATH_future)
    [["uid","d","t","x","y"]]
)
df_city_future.head()

,uid,d,t,x,y
0,17000,60,0,44.0,145.0
1,17000,60,1,44.0,145.0
2,17000,60,2,44.0,145.0
3,17000,60,3,44.0,145.0
4,17000,60,4,44.0,145.0


In [152]:
df_city_answer = (
    df_city
    # .loc[df_city["uid"].between(22000, 25000)] #B
    .loc[df_city["uid"].between(17000, 20000)] #C
    # .loc[df_city["uid"].between(3000, 6000)] #D
    .loc[df_city["d"] >= 60]
).drop(columns=["x","y"])

In [153]:
df_city_answer_with_predict = (
    pd.merge(
        df_city_answer,
        df_city_future,
        on = ["uid","d","t"],
        how = "left"
    )   
)

df_city_answer_with_predict["x"] = df_city_answer_with_predict["x"].astype(int)
df_city_answer_with_predict["y"] = df_city_answer_with_predict["y"].astype(int)

In [154]:
df_city_answer_with_predict.head()

,uid,d,t,x,y
0,17000,60,14,35,140
1,17000,60,15,35,140
2,17000,60,16,35,140
3,17000,60,17,35,140
4,17000,60,18,45,139


In [156]:
# 結果の確認
print(len(df_city_answer_with_predict["uid"].unique()))
print(df_city_answer_with_predict.shape)

(449308, 5)

### 提出データの出力

In [157]:
f"TeamName_city{city}_humob24.csv"

'TeamName_cityC_humob24.csv'

In [158]:
# Validator用にcsv出力
df_city_answer_with_predict.to_csv(f"TeamName_city{city}_humob24.csv",index=False)

In [159]:
# 提出用にcsv.gz出力
df_city_answer_with_predict.to_csv(f"TeamName_city{city}_humob24.csv.gz",index=False,compression="gzip")

#### ここまでをcityB,C,Dで繰り返して出力

### Validator

In [160]:
# cityB
!python3 geobleu/validator.py b df_cityB.csv TeamName_cityB_humob24.csv

Loading the submission file...

Checking the set of uid's...

Now loading the dataset file and comparing the submission data to it...

Validation finished without errors!


In [161]:
# cityC
!python3 geobleu/validator.py c df_cityC.csv TeamName_cityC_humob24.csv

Loading the submission file...

Checking the set of uid's...

Now loading the dataset file and comparing the submission data to it...

Validation finished without errors!


In [162]:
# cityD
!python3 geobleu/validator.py d df_cityD.csv TeamName_cityD_humob24.csv

Loading the submission file...

Checking the set of uid's...

Now loading the dataset file and comparing the submission data to it...

Validation finished without errors!
